In [1]:
import pandas as pd
from pandas import DataFrame
from docplex.mp.model import Model

import matplotlib.pyplot as plt
import random

from collections import defaultdict
from datetime import datetime, timedelta

### Read and Clean Data

In [3]:
excel_file = "Data Set - Class Schedule.xlsx"
sched_xlsx_file = pd.ExcelFile(excel_file)

# Load the sheets from the xlsx file into seperate dataframes

df_colleges_and_departments = pd.read_excel(sched_xlsx_file, 0)
df_course_catalog = pd.read_excel(sched_xlsx_file, 1)
df_Professors = pd.read_excel(sched_xlsx_file, 2)

df_Qualification = pd.read_excel(sched_xlsx_file, 3)
# Replace NAs with zeroes.
df_Qualification.fillna(0,inplace = True)

df_Courses_2025 = pd.read_excel(sched_xlsx_file, 4)
#Misspelling was tripping me up ...
df_Courses_2025.rename(columns={"Max Enrollement": "Max_Enrollment"}, inplace=True)

df_Classrooms = pd.read_excel(sched_xlsx_file, 5)




In [4]:
#Replace column name spaces with underscores for easier processing.

df_colleges_and_departments.columns = df_colleges_and_departments.columns.str.replace(" ","_")
df_course_catalog.columns = df_course_catalog.columns.str.replace(" ","_")
df_Professors.columns = df_Professors.columns.str.replace(" ","_")
df_Qualification.columns = df_Qualification.columns.str.replace(" ","_")
df_Courses_2025.columns = df_Courses_2025.columns.str.replace(" ","_")
df_Classrooms.columns = df_Classrooms.columns.str.replace(" ","_")

In [5]:
# base the duration of a class session on the number of credit hours
# of the course and the number of times a week the course meets

def calcSessionDuration(credit_hours, day_string):
    # session duration is 50 minutes * (number of credit hours/sessions per week)
    session_length = int((credit_hours / len(day_string.split('/'))) * 50)
    
    # Enforce session duration to be a multiple of 5 minutes because I thought having a class length of 62 minutes was weird.
    if(session_length % 5):
        session_length += 5 - (session_length % 5)
    
    return session_length

In [6]:
# Randomly create preferences for each course a professor is qualified for.

def generateProfessorPrefs():
    random.seed(3718)
    
    df_Qualification.set_index("Faculty_Name", inplace=True)

    # dictionary for randomly generated professor preferences.
    professor_prefs = {}
    
    for professor, row in df_Qualification.iterrows():
        # courses professor is qualified for
        qualified_courses = [course for course, val in row.items() if val == 1.0]
        
        # Generate random preferences
        preferences = list(range(1, len(qualified_courses) + 1))
        random.shuffle(preferences)
        
        professor_prefs[professor] = dict(zip(qualified_courses, preferences))

    return professor_prefs


In [7]:
# Get a block of time for a class based on it's duration.
def get_time_blocks(start_time, duration):
    blocks = []
    start_dt = datetime.strptime(start_time, "%H:%M")
    end_dt = start_dt + timedelta(minutes=duration)
    while start_dt < end_dt:
        blocks.append(start_dt.strftime("%H:%M"))
        start_dt += timedelta(minutes=30)
    return blocks


In [8]:
# Create a list of the possible start times between 8AM and 6PM

def generateStartTimes(start="08:00", end="18:00", interval=30):
    
    start_time = datetime.strptime(start, "%H:%M")
    end_time = datetime.strptime(end, "%H:%M")
    
    times = []  
    while start_time <= end_time:
        times.append(start_time.strftime("%H:%M"))
        start_time += timedelta(minutes=interval)
        
    return times
    

In [9]:
# Create entries for any course with multiuple sections.

def add_sections():
    expanded_rows = []
    for _, row in df_Courses_2025.iterrows():
        for section_num in range(1, row.Sections + 1):
            new_row = row.copy()
            new_row["Section_ID"] = section_num
            expanded_rows.append(new_row)

    return expanded_rows

In [10]:
# create dataframe with course information and duration of the class based on credithours/sessions per week
def merge_catalog_and_add_duration(df_course_sections):
    df_course_info = pd.merge(
        df_course_catalog,
        df_course_sections.drop(columns=["Course_Name"]),
        on=["Department_Code", "Course_Number"],
        how="inner"
    )
    df_course_info["Duration"] = df_course_info.apply(lambda row: calcSessionDuration(row.Credit_Hours, row.Day), axis = 1)
    
    return df_course_info


In [11]:
# Map the credit hours of each class and it's duration (credit_hours*50)/(num_sessions_per_week)
def create_duration_credit_maps(df_course_info):
     return {(f"ENTOM_{row.Course_Number}", row.Section_ID): row.Duration for _, row in df_course_info.iterrows()}, {(f"ENTOM_{row.Course_Number}", row.Section_ID): row.Credit_Hours for _, row in df_course_info.iterrows()}

In [12]:
#Find all valid timeslots for each course based on it's duration and the valid start and end times.

def find_valid_timeslots(start_times,df_course_info):
    
    valid_timeslots = {} 
    for _, row in df_course_info.iterrows():
        course = f"ENTOM_{row.Course_Number}"
        section = row.Section_ID  
        duration = row.Duration 
    
        valid_slots = []
        for t in start_times:
            t_start = datetime.strptime(t, "%H:%M")
            t_end = t_start + timedelta(minutes=duration)
    
            # 6:00 PM cutoff
            if t_end <= datetime.strptime("18:00", "%H:%M"):
                valid_slots.append(t)
    
        valid_timeslots[(course, section)] = valid_slots
    return valid_timeslots


In [13]:
# ensure that a room is large enough to hold the class at maximum capacity.

def validRoom(row, df_Classrooms):
    valid = False
    for room in df_Classrooms["Classroom"]:
        room_capacity = df_Classrooms.loc[df_Classrooms["Classroom"] == room, "Capacity"].values[0]
        if room_capacity >= row.Max_Enrollment:
            valid = True
            break
    return valid

In [14]:
def create_decision_variables(df_course_info, df_Classrooms, valid_timeslots, professors):

    x = {}
    for _, row in df_course_info.iterrows():
        course = f"ENTOM_{row.Course_Number}"
        section = row.Section_ID
        duration = row.Duration
        found_any = False
    
        # Check: Is there at least one room that can hold this class?
        if not validRoom(row, df_Classrooms):
            continue  # Skip to next course-section
    
        # check professor + room + time combinations
        for professor in professors:
            if course not in prof_pref.get(professor, {}):
                continue
            for room_row in df_Classrooms.itertuples(index=False):
                room = room_row.Classroom
                room_capacity = room_row.Capacity
    
                if room_capacity < row.Max_Enrollment:
                    continue
                for time in valid_timeslots.get((course, section), []):
                    found_any = True
                    x[(course, section, professor, room, time)] = model.binary_var()
    return x

In [15]:
# Constraint to ensure Professor is not assigned more credit hours than the specified Workload limit

def professor_workload_constraint(x, model, professors, cred_map, prof_workload, df_professors):
    for professor in professors:
        assigned_credits = []
        for (course, section, prof, room, time), var in x.items():
            if prof == professor:
                credits = cred_map.get((course, section), 0)
                assigned_credits.append(credits * var)
    
        max_allowed = prof_workload.get(professor, 0)
        if assigned_credits:
            model.add_constraint(model.sum(assigned_credits) <= max_allowed, f"MaxCredits_{professor}")


In [16]:
# Parse out the list of days the course is held ("M","TU", "W", "TH", "F") and map teh list to the course

def create_course_day_map(df_course_info):
    return {
        (f"ENTOM_{row.Course_Number}", row.Section_ID): row.Day.split("/")
        for _, row in df_course_info.iterrows()
    }

In [17]:
# Do not allow the Professor to be assigned to times that overlap.
def professor_overlap_class_constraint(x, model, professors, start_times, course_day_map):
    for professor in professors:
        for time in start_times:
            for day in ["M", "TU", "W", "TH", "F"]:
                overlapping_vars = [
                    var
                    for (course, section, prof, room, t), var in x.items()
                    if prof == professor
                    and t == time
                    and day in course_day_map.get((course, section), [])
                ]
                if overlapping_vars:
                    model.add_constraint(model.sum(overlapping_vars) <= 1, 
                                         ctname=f"prof_time_conflict_{professor}_{day}_{time.replace(':','')}")


In [18]:
# Add constraint for any additional sections (> 1)
def course_number_and_section_constraint(x, model, df_Courses_2025):
    for course, section in df_Courses_2025[["Course_Number", "Sections"]].drop_duplicates().itertuples(index=False):
        course_id = f"ENTOM_{course}"

    # Collect all variables for this course section
    vars_for_section = [
        x[key]
        for key in x
        if key[0] == course_id and key[1] == section
    ]

    if vars_for_section:  # Only add constraint if variables exist
        model.add_constraint(model.sum(vars_for_section) == 1, ctname=f"assign_once_{course_id}_sec{section}")
    else:
        print(f" Warning: No variables found for {course_id}, section {section}")
    

In [19]:
# Do not allow a room to be used by more than one class at a time. 
def room_overlap_constraint(x, model, df_Classrooms):
    for room in df_Classrooms["Classroom"]:
        for day in ["M", "TU", "W", "TH", "F"]:
            for time in start_times:
                overlapping_vars = []
                for key, var in x.items():
                    course, section, professor, rm, start = key
                    if rm == room and day in course_day_map.get((course, section), []):
                        occupied = get_time_blocks(start, dur_map[(course, section)])
                        if time in occupied:
                            overlapping_vars.append(var)
                model.add_constraint(model.sum(overlapping_vars) <= 1)
    

In [20]:
# each section should be assigned only once.
def course_sectionID_constraint(x, model, df_course_info):
    for (course, section) in df_course_info[["Course_Number", "Section_ID"]].drop_duplicates().itertuples(index=False):
        course_id = f"ENTOM_{course}"
        vars_for_section = [
            x[key] for key in x
            if key[0] == course_id and key[1] == section
        ]
        if vars_for_section:
            model.add_constraint(model.sum(vars_for_section) == 1, ctname=f"assign_once_{course_id}_sec{section}")

In [21]:
# Prevent professor from teaching multiple sections of the same course.  
def professor_sectionID_constraint(x, model, df_course_info):
    # Prevent professors from teaching multiple sections of the same course
    for professor in professors:
        for course in df_course_info["Course_Number"].unique():
            course_name = f"ENTOM_{course}"
            vars_for_course = [
                var for (c, s, p, r, t), var in x.items()
                if c == course_name and p == professor
            ]
            if vars_for_course:
                model.add_constraint(model.sum(vars_for_course) <= 1,
                                     ctname=f"limit_{professor}_{course_name}")

In [22]:
def create_objective_function(model, x, prof_pref):
    objective_terms = []
    for key, var in x.items():
        course, section, professor, room, time = key
        preference = prof_pref.get(professor, {}).get(course, None)
        if preference is not None:
            objective_terms.append(preference * var)
    
    model.minimize(model.sum(objective_terms))

#### Main

In [24]:
model = Model("FullClassScheduler")

# If any courses have more than one section, create a row for them
df_Courses_2025_sections = pd.DataFrame(add_sections())

# Add new Sections and duration of each class session
df_course_info = merge_catalog_and_add_duration(df_Courses_2025_sections)

course_day_map = create_course_day_map(df_course_info)

# Create mapping for duration and credit hours for each class
dur_map, cred_map = create_duration_credit_maps(df_course_info)

# Create a list of the valid start times for all courses based on their duration 
start_times = generateStartTimes()
valid_timeslots = find_valid_timeslots(start_times, df_course_info)

# Randomly Generate Professor Preferences
prof_pref = generateProfessorPrefs()

# List of all Professors
professors = df_Professors["Faculty_Name"].unique()

#Limit of Professor Credit hours
prof_workload = df_Professors.set_index("Faculty_Name")["Workload_Credit_Hours"].to_dict()

# Decision Variables
x = create_decision_variables(df_course_info, df_Classrooms, valid_timeslots, professors)

# Add Constraints
professor_workload_constraint(x, model, professors, cred_map, prof_workload, df_Professors)
professor_overlap_class_constraint(x, model, professors, start_times, course_day_map)
course_number_and_section_constraint(x, model, df_Courses_2025)
room_overlap_constraint(x, model, df_Classrooms)
course_sectionID_constraint(x, model, df_course_info)
professor_sectionID_constraint(x, model, df_course_info)

# Objective
create_objective_function(model, x, prof_pref)

In [25]:
solution = model.solve(log_output = True)

Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 2745 rows and 9399 columns.
MIP Presolve modified 1078 coefficients.
Reduced MIP has 930 rows, 20276 columns, and 118366 nonzeros.
Reduced MIP has 20276 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.09 sec. (165.14 ticks)
Found incumbent of value 189.000000 after 0.17 sec. (301.40 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 930 rows, 20276 columns, and 118366 nonzeros.
Reduced MIP has 20276 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.05 sec. (113.51 ticks)
Probing time = 0.02 sec. (13.31 ticks)
Clique table members: 1223.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.02 sec. (46.08 ticks)

        Nodes                                        

In [26]:
if solution:
    print("Solution found!")
else:
    print("No feasible solution.")


Solution found!


In [27]:
for key, var in x.items():
    if var.solution_value > 0.5:
        course, section, professor, room, time = key
        days = course_day_map.get((course, section), [])
        day_str = "/".join(days)
        duration = dur_map.get((course, section), 75)
        
        # Calculate end time
        start_dt = datetime.strptime(time, "%H:%M")
        end_dt = start_dt + timedelta(minutes=duration)
        end_time = end_dt.strftime("%H:%M")


In [28]:
schedule_data = []
professor_workload = defaultdict(int)

for key, var in x.items():
    if var.solution_value > 0.5:
        course, section, professor, room, time = key
        days = course_day_map.get((course, section), [])
        day_str = "/".join(days)
        duration = dur_map.get((course, section), 75)

        # Get credit hours for this course section
        credit_hours = df_course_info[
            (df_course_info["Course_Number"] == int(course.split("_")[1])) &
            (df_course_info["Section_ID"] == section)
        ]["Credit_Hours"].values[0]

        # Track workload
        professor_workload[professor] += credit_hours

        # Calculate end time
        start_dt = datetime.strptime(time, "%H:%M")
        end_dt = start_dt + timedelta(minutes=duration)
        end_time = end_dt.strftime("%H:%M")

        schedule_data.append({
            "Course": course,
            "Section": section,
            "Days": day_str,
            "Start Time": time,
            "End Time": end_time,
            "Professor": professor,
            "Room": room
        })

# Add workload to the DataFrame
schedule_df = pd.DataFrame(schedule_data)
schedule_df["Workload (Credit Hours)"] = schedule_df["Professor"].map(professor_workload)

# Export to Excel
schedule_df.to_excel("final_schedule.xlsx", index=False)
print("Excel file saved: final_schedule.xlsx")


Excel file saved: final_schedule.xlsx


In [55]:
schedule_df


,Course,Section,Days,Start Time,End Time,Professor,Room,Workload (Credit Hours)
0,ENTOM_300,1,M/W/F,17:00,17:50,Weston Opitz,1052,9
1,ENTOM_301,1,TU/TH,08:00,09:15,James F. Campbell,1073,8
2,ENTOM_301,2,TU/TH,16:30,17:45,Erin Scully,1052,3
3,ENTOM_301,3,TU/TH,12:00,13:15,Frank H. Arthur,1052,6
4,ENTOM_305,1,TU/TH,14:00,15:15,Weston Opitz,1073,9
5,ENTOM_305,2,TU/TH,08:00,09:15,Frank H. Arthur,1052,6
6,ENTOM_305,3,TU/TH,16:30,17:45,Raymond Cloyd,1066,7
7,ENTOM_306,1,M/W/F,08:00,08:50,Jeremy L. Marshall,1066,9
8,ENTOM_350,1,M/W,11:30,12:45,Brian Spiesman,1073,6
9,ENTOM_589,1,M/W/F,17:00,17:50,Brian Spiesman,1029,6
